# TensorBoard Tutorial

In [1]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np

x_data = np.array([
    [0,0],
    [1,0],
    [1,1],
    [0,0],
    [0,0],
    [0,1]
])

y_data = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1],
    [1,0,0],
    [1,0,0],
    [0,0,1]
])


### Neural Network 구성

In [2]:
global_step = tf.Variable(0, trainable=False, name='global_step')

학습에 직접적으로 사용하지 않고 학습 횟수에 따라서 단순히 증가시킬 변수를 만든다.

In [3]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

첫번째 Weight의 Dimension은 Feature의 개수 2, 히든 레이어의 노드 개수를 10개로 가정하고 [ 2, 10 ] 으로 정의한다.

In [4]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
    L1 = tf.nn.relu(tf.matmul(X,W1))
    
    tf.summary.histogram("X",X)
    tf.summary.histogram("Weights",W1)

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10,20], -1., 1.))
    L2 = tf.nn.relu(tf.matmul(L1,W2))
    
    tf.summary.histogram("Weights",W2)

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20,3], -1.,1.))
    model = tf.matmul(L2,W3)
    
    tf.summary.histogram("Weights",W3)
    tf.summary.histogram("Model",model)

tf.name_scope로 묶은 블럭은 텐서보드에서 한 레이어안에 표현이 된다.

In [5]:
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    tf.summary.scalar('cost',cost)

tf.summary.scalar를 이용해 수집하고 싶은 값들 지정이 가능하다.

### Neural Network 학습

In [6]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-200


In [7]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs',sess.graph)

텐서보드에서 표시하기 위한 텐서들을 수집하고, 그래프와 텐서값들을 저장할 디렉토리를 설정한다.  
이후에 이 로그파일은, tensorboard --logdir=./logs 명령어를 사용하여 서버를 연뒤,  
http://localhost:6006 에서 확인할 수 있다.

In [8]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    if (step+1) % 10 ==0:
        print('Step: %d, ' % sess.run(global_step),
              'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    summary = sess.run(merged, feed_dict={X:x_data, Y:y_data})
    writer.add_summary(summary,global_step=sess.run(global_step))
    
saver.save(sess, './model/dnn.ckpt',global_step=global_step)

('Step: 210, ', 'Cost: 0.550')
('Step: 220, ', 'Cost: 0.550')
('Step: 230, ', 'Cost: 0.549')
('Step: 240, ', 'Cost: 0.549')
('Step: 250, ', 'Cost: 0.549')
('Step: 260, ', 'Cost: 0.549')
('Step: 270, ', 'Cost: 0.549')
('Step: 280, ', 'Cost: 0.549')
('Step: 290, ', 'Cost: 0.549')
('Step: 300, ', 'Cost: 0.549')


'./model/dnn.ckpt-300'

In [9]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y,1)
print('Prediction:', sess.run(prediction, feed_dict={X: x_data}))
print('Target:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

('Prediction:', array([0, 1, 2, 0, 0, 2]))
('Target:', array([0, 1, 2, 0, 0, 2]))
Accuracy: 100.00


### 결과 확인
- 0 : 기타
- 1 : 포유류
- 2 : 조류